In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

work_path = '/content/gdrive/MyDrive/PaperRecommendation/finalize_code_folder/'
checkpoint_path = work_path + "checkpoint/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!wget https://raw.githubusercontent.com/dangnhiGithub/Paper-Recommendation-Systems---AEI-IEA-2021/main/function_01.py
%run function_01.py

--2021-04-16 16:25:48--  https://raw.githubusercontent.com/dangnhiGithub/Paper-Recommendation-Systems---AEI-IEA-2021/main/function_01.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12741 (12K) [text/plain]
Saving to: ‘function_01.py.1’

function_01.py.1    100%[===================>]  12.44K  --.-KB/s    in 0s      

2021-04-16 16:25:49 (25.2 MB/s) - ‘function_01.py.1’ saved [12741/12741]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class preprocess_text():
    def preprocess_Abstract(self, text):
        if type(text) == str:
            text = text.lower()
            text = re.sub("[\d+]", "", text)
            text = re.sub("[^a-z]", " ", text)

            filtered_sentence = [] 
            for w in text.split(" "): 
                if w not in stop_words and w not in stop:
                    filtered_sentence.append(w)
            text = " ".join(filtered_sentence)
            text = text.strip()
            text = re.sub("[ ]{2,}", " ", text)
        return text

    def preprocess_Title(self, text):
        return self.preprocess_Abstract(text)

    def preprocess_Aims(self, text):
        return self.preprocess_Abstract(text)
    
    def preprocess_Keywords(self, text):
        return self.preprocess_Abstract(text)

In [ ]:
tool_preprocess = preprocess_tool(preprocess_text())

In [ ]:
# import multiprocessing

# print("Preprocessing train ....")
# data_train = pd.read_csv(work_path + "data_splited_train.csv", encoding = "ISO-8859-1")
# data_train = tool_preprocess.get_preprocessed_data(
#     data_train,
#     preprocess_columns = ["title", "abstract", "keywords"],
#     preprocessing_type = ["Title", "Abstract", "Keywords"],
#     keep_columns = ["itr"],
#     n_jobs=multiprocessing.cpu_count()
#     )

# print("Preprocessing validate ....")
# data_validate = pd.read_csv(work_path + "data_splited_validate.csv", encoding = "ISO-8859-1")
# data_validate = tool_preprocess.get_preprocessed_data(
#     data_validate,
#     preprocess_columns = ["title", "abstract", "keywords"],
#     preprocessing_type = ["Title", "Abstract", "Keywords"],
#     keep_columns = ["itr"],
#     n_jobs=multiprocessing.cpu_count()
#     )

# print("Preprocessing test ....")
# data_test = pd.read_csv(work_path + "data_origin_test.csv", encoding = "ISO-8859-1")
# data_test = tool_preprocess.get_preprocessed_data(
#     data_test,
#     preprocess_columns = ["title", "abstract", "keywords"],
#     preprocessing_type = ["Title", "Abstract", "Keywords"],
#     keep_columns = ["itr"],
#     n_jobs=multiprocessing.cpu_count()
#     )

# print("Preprocessing aims ....")
# data_aims = pd.read_csv(work_path + "aims&scopes.csv", encoding = "ISO-8859-1")
# data_aims = tool_preprocess.get_preprocessed_data(
#     data_aims,
#     preprocess_columns = ["aims"],
#     preprocessing_type = ["Aims"],
#     keep_columns = ["itr"],
#     n_jobs=multiprocessing.cpu_count()
#     )

# data_train['Label'] = labelling_data(data_train["itr"], data_aims["itr"])
# data_validate['Label'] = labelling_data(data_validate["itr"], data_aims["itr"])
# data_test['Label'] = labelling_data(data_test["itr"], data_aims["itr"])

# data_train.to_csv(checkpoint_path + "01_train.csv", index=False)
# data_validate.to_csv(checkpoint_path + "01_validate.csv", index=False)
# data_test.to_csv(checkpoint_path + "01_test.csv", index=False)
# data_aims.to_csv(checkpoint_path + "01_aims.csv", index=False)



data_train = pd.read_csv(checkpoint_path + "01_train.csv", encoding = "ISO-8859-1")
data_validate = pd.read_csv(checkpoint_path + "01_validate.csv", encoding = "ISO-8859-1")
data_test = pd.read_csv(checkpoint_path + "01_test.csv", encoding = "ISO-8859-1")
data_aims = pd.read_csv(checkpoint_path + "01_aims.csv", encoding = "ISO-8859-1")

data_train.fillna("", inplace=True)
data_validate.fillna("", inplace=True)
data_test.fillna("", inplace=True)
data_aims.fillna("", inplace=True)

In [ ]:
tool_NLP = NLP_tool()

corpus = (data_train['Title'] + ' ' + data_train['Abstract'] + " " + data_train['Keywords']).tolist()
corpus = corpus + data_aims['Aims'].tolist()
tokenizer = tool_NLP.build_tokenizer(corpus, vocabulary_size=None)
save_parameter(tokenizer, save_file=checkpoint_path+"01_tokenizer.pickle")

# tokenizer = load_parameter(load_file=checkpoint_path+"01_tokenizer.pickle")

In [ ]:
tool_NLP.download_FastText_pretrained()
fasttext_embedding_dict = tool_NLP.build_fasttext_embedding_dict()
embedding_matrix, zero_tokens = tool_NLP.build_embedding_matrix_FastText(tokenizer, fasttext_embedding_dict)
save_parameter([embedding_matrix, zero_tokens], save_file=checkpoint_path + "01_embedding_matrix.pickle")

# [embedding_matrix, zero_tokens] = load_parameter(load_file=checkpoint_path + "01_embedding_matrix.pickle")


Building embedding matrix .....


In [ ]:
if tokenizer.num_words == None:
    vocabulary_size = len(tokenizer.word_index)
else:
    vocabulary_size = tokenizer.num_words

print(vocabulary_size)

display(zero_tokens)

288743


In [ ]:
len(zero_tokens)

138030

In [ ]:
from scipy.spatial import distance
from scipy import sparse

class similarity_tool():
    def __init__(self,embedding_matrix, zero_tokens,
                 vocabulary_size = 50000, embedding_dimension=300,
                 n_jobs=4):
        self.embedding_matrix = embedding_matrix
        self.embedding_dimension = embedding_dimension
        self.zero_tokens = zero_tokens
        self.n_jobs = n_jobs
    
    def calculate_centroid_data(self, tokenized_data):
        '''-Parameter:
              tokenized_data(numpy array):
           -Return:
              (numpy array): matrix A each row is centroid vector respective tokenized_data points. 
        '''######
        # nPoints = tokenized_data.shape[0]
        # embedding_dimension = self.embedding_matrix.shape[1]
        
        with Pool(self.n_jobs) as p:
            centroid_data = p.map(self.subfunction_01_calculate_centroid_vector, tokenized_data)
        centroid_data = np.array(centroid_data)
        return centroid_data

    def subfunction_01_calculate_centroid_vector(self, tokenized_doc_i):
        tokens = tokenized_doc_i[tokenized_doc_i != 0]
        tokens = [token for token in tokens if token not in self.zero_tokens]
        if len(tokens):
            centroid_vector = np.mean(self.embedding_matrix[tokens], axis=0)
        else:
            centroid_vector = np.zeros(self.embedding_matrix.shape[1])
        return centroid_vector
    
    def calculate_matrix_cosine(self, centroid_data_01, centroid_data_02):
        '''-Parameter:
              centroid_data_01(numpy array):
              centroid_data_02(numpy array):
           -Return:
              (numpy array): matrix A(i,j) which (i,j) is cosine distance of data_(i)th and aims_(j)th 
        '''######
        self.centroid_data_aims = centroid_data_02
        self.n_aims = centroid_data_02.shape[0]
       
        with Pool(self.n_jobs) as p:
            output_matrix = p.map(self.subfunction_02_cosine, centroid_data_01)
        output_matrix = np.asarray(output_matrix)
        return output_matrix

    def subfunction_02_cosine(self,centroid_x):
        output = np.zeros(self.n_aims) 
        if np.count_nonzero(centroid_x):
            for i in range(self.n_aims):
                output[i] = distance.cosine(centroid_x, self.centroid_data_aims[i])
        return output

In [ ]:
tool_similarity = similarity_tool(embedding_matrix, zero_tokens)

#####
X_train = tool_NLP.tokenize_data(data_train['Title']
                                 +' '+data_train['Abstract']
                                 +' '+data_train['Keywords'],
                                 tokenizer)
centroid_train = tool_similarity.calculate_centroid_data(X_train)
print(X_train.shape)

X_validate = tool_NLP.tokenize_data(data_validate['Title']
                                    +' '+data_validate['Abstract']
                                    +' '+data_validate['Keywords'], 
                                    tokenizer)
centroid_validate = tool_similarity.calculate_centroid_data(X_validate)
print(X_validate.shape)


X_test = tool_NLP.tokenize_data(data_test['Title']
                                +' '+data_test['Abstract']
                                +' '+data_test['Keywords'],
                                tokenizer)
centroid_test = tool_similarity.calculate_centroid_data(X_test)
print(X_test.shape)

tokenized_aims = tool_NLP.tokenize_data(data_aims['Aims'], tokenizer)
centroid_aims = tool_similarity.calculate_centroid_data(tokenized_aims)
print(tokenized_aims.shape)

(298317, 637)
(33147, 426)
(83048, 497)
(351, 640)


In [ ]:
#####

# # cosine_train
# t1 = time.time()
# cosine_train = tool_similarity.calculate_matrix_cosine(centroid_train, centroid_aims)
# t2 = time.time()
# print("Calculate cosine_train runtime: %.f s "%(t2 - t1))
# save_parameter(cosine_train, save_file=checkpoint_path + "pickle/01_cosine_train.pickle")

# # cosine_validate
# t1 = time.time()
# cosine_validate = tool_similarity.calculate_matrix_cosine(centroid_validate, centroid_aims)
# t2 = time.time()
# print("Calculate cosine_validate runtime: %.f s "%(t2 - t1))
# save_parameter(cosine_validate, save_file=checkpoint_path + "pickle/01_cosine_validate.pickle")

# #cosine_test
# t1 = time.time()
# cosine_test = tool_similarity.calculate_matrix_cosine(centroid_test, centroid_aims)
# t2 = time.time()
# print("Calculate cosine_test runtime: %.f s "%(t2 - t1))
# save_parameter(cosine_test, save_file=checkpoint_path + "pickle/01_cosine_test.pickle")


cosine_train = load_parameter(load_file=checkpoint_path + "pickle/01_cosine_train.pickle")
cosine_validate = load_parameter(load_file=checkpoint_path + "pickle/01_cosine_validate.pickle")
cosine_test = load_parameter(load_file=checkpoint_path + "pickle/01_cosine_test.pickle")

print(cosine_train.shape)
print(cosine_validate.shape)
print(cosine_test.shape)

Calculate cosine_train runtime: 2242 s 
Calculate cosine_validate runtime: 263 s 
Calculate cosine_test runtime: 632 s 


In [ ]:
from sklearn.preprocessing import OneHotEncoder
num_labels = len(data_aims)

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit( np.array(range(num_labels)).reshape(-1,1) )
Y_train = onehot_encoder.transform(np.array(data_train['Label']).reshape((-1,1)))
Y_validate = onehot_encoder.transform(np.array(data_validate['Label']).reshape((-1,1)))
Y_test = onehot_encoder.transform(np.array(data_test['Label']).reshape((-1,1)))

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import functools     # special library for loss function in keras


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dropout, Flatten, Concatenate, GlobalMaxPooling1D, Activation, GRU, MaxPooling1D
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = "top3"

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = "top5"

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = "top10"

In [ ]:
corpus_train = (
    data_train['Title']  
    + " " + data_train['Abstract']
    + " " + data_train['Keywords']
    )
corpus_validate = (
    data_validate['Title']  
    + " " + data_validate['Abstract']
    + " " + data_validate['Keywords']
    )
corpus_test = (
    data_test['Title']  
    + " " + data_test['Abstract']
    + " " + data_test['Keywords']
    )

X_train = tool_NLP.tokenize_data(corpus_train, tokenizer, maxlen = 700)
X_validate = tool_NLP.tokenize_data(corpus_validate, tokenizer, maxlen = 700)
X_test = tool_NLP.tokenize_data(corpus_test, tokenizer, maxlen = 700)

print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

(298317, 700)
(33147, 700)
(83048, 700)


# Run 1 without Aims

In [ ]:
shape_01 = X_train.shape[1]

inputs_01 = Input(shape = (shape_01,) )    # data tokenized max length
embedding_01 = Embedding(embedding_matrix.shape[0], 300, 
                         input_length = shape_01 ,
                         weights = [embedding_matrix],
                         trainable = False)(inputs_01)

conv1d_01 = Conv1D(filters=1800, kernel_size=2, activation='relu')(embedding_01)
pool_01 = GlobalMaxPooling1D()(conv1d_01)
flat_01 = Flatten()(pool_01)
dense_01 = Dense(1000, activation='relu')(flat_01)
drop_01 = Dropout(0.2)(dense_01)

                       
outputs = Dense(num_labels, activation='softmax')(drop_01)    # aims length is also number label
model = Model(inputs = inputs_01, outputs = outputs)

model.compile(loss='categorical_crossentropy', 
              optimizer="Adam",
              metrics=["accuracy",top3_acc,top5_acc,top10_acc])


print(model.summary())

mc = keras.callbacks.ModelCheckpoint(filepath = checkpoint_path + "weight/02_noAims_TAK.h5", 
                                     monitor='val_accuracy',
                                     mode='max', 
                                     verbose=1, 
                                     save_best_only=True)
# model = load_model('filename')
history = model.fit(X_train, Y_train,
                    batch_size=512,
                    epochs=30,
                    validation_data=(X_validate, Y_validate),
                    callbacks=[mc])

matrix_predict = model.predict(X_test)
Y_predict = np.argsort(matrix_predict, axis=1)[:,::-1]
GroundTruth = np.array(data_test['Label'])
for top_k in [1,3,5,10]:
    count = 0
    nPoints = len(GroundTruth)
    for i in range(nPoints):
        if GroundTruth[i] in Y_predict[i, 0:top_k]:
            count += 1
    print("top {} : {}".format(top_k,count/nPoints))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 700)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 700, 300)          95110200  
_________________________________________________________________
conv1d (Conv1D)              (None, 699, 1800)         1081800   
_________________________________________________________________
global_max_pooling1d (Global (None, 1800)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1800)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              1801000   
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0     

# Run 2 Aims & Scopes

In [ ]:
shape_1 = X_train.shape[1]
shape_2 = cosine_train.shape[1]


inputs1 = Input(shape=(shape_1,) )    # data tokenized max length
inputs2 = Input(shape=(shape_2,))

#cosine
batch2_1 = tf.keras.layers.BatchNormalization()(inputs2)
dense2_1 = Dense(1500,activation="relu")(batch2_1)
drop2_1 = tf.keras.layers.Dropout(0.4)(dense2_1)
dense2_2 = Dense(1000,activation="relu")(drop2_1)
drop2_2 = tf.keras.layers.Dropout(0.4)(dense2_2)
dense2_3 = Dense(500,activation="relu")(drop2_2)
drop2_3 = tf.keras.layers.Dropout(0.4)(dense2_3)
#flow_main

embedding1 = Embedding(embedding_matrix.shape[0], 300, 
                      input_length = shape_1 ,
                      weights = [embedding_matrix],
                      trainable = False)(inputs1)

conv1 = Conv1D(filters=1800, kernel_size=2, activation='relu')(embedding1)
drop1_1 = Dropout(0.3)(conv1)
pool1_1 = GlobalMaxPooling1D()(drop1_1)

dense1_1 = Dense(1000, activation='relu')(pool1_1)
drop1_2 = Dropout(0.3)(dense1_1)
dense1_2 = Dense(500,activation="relu")(drop1_2)
drop1_3 = Dropout(0.3)(dense1_2)

concat = tf.keras.layers.Concatenate(axis=1)([drop2_3, drop1_3])

dense = Dense(500,activation="relu")(concat)
drop = Dropout(0.3)(dense)
                       
outputs = Dense(num_labels, activation='softmax')(drop)    # aims length is also number label
model = Model(inputs=[inputs1,inputs2], outputs=outputs)

model.compile(loss='categorical_crossentropy', 
              optimizer="Adam",
              metrics=["accuracy",top3_acc,top5_acc,top10_acc])

print(model.summary())

mc = keras.callbacks.ModelCheckpoint(filepath = checkpoint_path + "weight/02_Aims_TAK.h5", 
                                     monitor='val_accuracy',
                                     mode='max', 
                                     verbose=1, 
                                     save_best_only=True)

# model = load_model('filename')
history = model.fit([X_train, cosine_train], Y_train,
                    batch_size=512,
                    epochs=30,
                    validation_data=([X_validate, cosine_validate], Y_validate),
                    callbacks=[mc])


matrix_predict = model.predict([X_test, cosine_test])
Y_predict = np.argsort(matrix_predict, axis=1)[:,::-1]
GroundTruth = np.array(data_test['Label'])
for top_k in [1,3,5,10]:
    count = 0
    nPoints = len(GroundTruth)
    for i in range(nPoints):
        if GroundTruth[i] in Y_predict[i, 0:top_k]:
            count += 1
    print("top {} : {}".format(top_k,count/nPoints))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 700)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 351)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 700, 300)     95110200    input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 351)          1404        input_2[0][0]                    
______________________________________________________________________________________________